In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re
from shapely.geometry import Point, Polygon

In [2]:
# ejemplo con corte del 30 de nov
path = '../data/original/'
file= 'Bit_Tecs_SEFADER_29Nov2024.csv' 
output_file_name = '../data/corte_30_nov.csv'

In [3]:
## Archivos adicionales 
#esta es la última versión de relacion tecnicos-programa-region
programa_file = '../data/inputs/relacion_tecnicos_programa_version_noviembre.csv'
# capa oax
regiones_oax_file = '../data/regiones_oax.geojson'


In [4]:
corte_nov = pd.read_csv(path+file)

/tmp/ipykernel_6987/999522281.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  corte_nov = pd.read_csv(path+file)


In [5]:
corte_nov.shape
# (22392, 35)

(22392, 35)

In [6]:
corte_nov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22392 entries, 0 to 22391
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID.de.bitacora                  22392 non-null  object 
 1   Tipo.de.superficie              22392 non-null  object 
 2   ID.del.productor                1729 non-null   object 
 3   Nombre.Productor                22392 non-null  object 
 4   CURP                            1729 non-null   object 
 5   ID.parcela                      862 non-null    object 
 6   ID.del.sitio                    1606 non-null   object 
 7   Nombre.de.la.Parcela            22392 non-null  object 
 8   Estado.areas                    22392 non-null  object 
 9   Municipio.areas                 22391 non-null  object 
 10  Coordenadas.areas               22392 non-null  object 
 11  Tipo.de.bitacora                22392 non-null  object 
 12  Anio.Bitacora                   

In [7]:
corte_nov[['Bitacora.registrada.por','ID usuario']].head(3)

,Bitacora.registrada.por,ID usuario
0,UrielGonzales_PSUR,users-2023-0606-0940-a4X8a-AGOSM
1,CarlosCervantes_PSUR,users-2023-0601-1101-TwTJg-YwmbF
2,SalvadorPelaez_PSUR,users-2023-0601-1041-sMTkP-CDPAR


In [8]:
corte_nov['Anio.Bitacora'].value_counts()

Anio.Bitacora
PV_2024         20258
OI_2023_2024     2134
Name: count, dtype: int64

In [9]:
corte_nov['Tipo.de.bitacora'].value_counts()

Tipo.de.bitacora
AI simplificada    21530
MODULO               435
AE                   414
AI                    13
Name: count, dtype: int64

In [10]:
corte_nov.iloc[:,24:34].head(3)

,1_Agricultura de conservacion,2_Arreglos de siembra,3_Comercializacion,4_Conservacion de agua,5_Conservacion del suelo,6_Fertilizacion,7_Manejo integrado de plagas,8_Tecnologias de poscosecha,9_Variedades mejoradas,10_Otra
0,0,0,0,0,1,0,0,0,0,0
1,0,1,0,0,0,1,1,0,0,0
2,1,0,0,0,0,0,0,0,0,0


In [11]:
def clean_names_prefix(list_columns:list) -> list:
    new_names = []
    for col in list_columns:
        split_str = col.split('_')
        output = '_'.join(split_str[1:])
        new_names.append(output)
    return new_names

In [12]:
columns_name =  list(corte_nov.columns)
tecno_cols = columns_name[24:34]

In [13]:
tecno_cols

['1_Agricultura de conservacion',
 '2_Arreglos de siembra',
 '3_Comercializacion',
 '4_Conservacion de agua',
 '5_Conservacion del suelo',
 '6_Fertilizacion',
 '7_Manejo integrado de plagas',
 '8_Tecnologias de poscosecha',
 '9_Variedades mejoradas',
 '10_Otra']

In [14]:
names_tecno = clean_names_prefix(tecno_cols)

In [15]:
corte_nov.rename(columns=dict(zip(tecno_cols,names_tecno)),inplace=True)

In [16]:
# validar no nulos para crear una pk 
corte_nov['ID.de.bitacora'].isnull().sum()

0

In [17]:
corte_nov['Tipo.de.superficie'].isnull().sum()

0

In [18]:
corte_nov['Tipo.de.bitacora'].isnull().sum()

0

In [19]:
corte_nov['ID.de.bitacora'].nunique()
# 22225

22225

In [20]:
#en caso de contar nulos no es posible crearla
corte_nov['primary_k'] = corte_nov['ID.de.bitacora'] + corte_nov['Tipo.de.superficie'] + corte_nov['Tipo.de.bitacora']

In [21]:
corte_nov['primary_k'].nunique()
# 22392
# la pk debe ser de la misma longitud del dataset
# RangeIndex: 22392 entries, 0 to 22391

22392

In [22]:
# asignación del programa de acuerdo al id de usuario quien creo la bitacora 

programa = pd.read_csv(programa_file)

In [23]:
programa.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Bitacora.registrada.por  239 non-null    object
 1   NOMBRE DEL USUARIO       239 non-null    object
 2   id                       239 non-null    object
 3   user_name                239 non-null    object
 4   first_name               239 non-null    object
 5   last_name                239 non-null    object
 6   full_name_tec            239 non-null    object
 7   Nombre del técnico       175 non-null    object
 8   Programa                 195 non-null    object
 9   Región de atención       205 non-null    object
 10  respuesta_hub            44 non-null     object
dtypes: object(11)
memory usage: 20.7+ KB


In [24]:
programa.head(2)

,Bitacora.registrada.por,NOMBRE DEL USUARIO,id,user_name,first_name,last_name,full_name_tec,Nombre del técnico,Programa,Región de atención,respuesta_hub
0,AbrahamTejeda_PSUR,Abraham Tejeda Galmich,users-2023-0724-1048-KAgLB-hfT0v,AbrahamTejeda_PSUR,Abraham,Tejeda Galmich,Abraham Tejeda Galmich,Abraham Tejeda Galmich,Abasto Seguro de Maíz,Costa,NaN
1,AidaLopez_PSUR,Aida López Cruz,users-2023-0601-1113-PJSxf-jigpv,AidaLopez_PSUR,Aida,López Cruz,Aida López Cruz,Aida López Cruz,Autosuficiencia Alimentaria,Mixteca,NaN


In [25]:
programa['Bitacora.registrada.por'].nunique()
# 239
# RangeIndex: 239 entries, 0 to 238

239

In [26]:
#add progrma
corte_nov=corte_nov.merge(programa[['Bitacora.registrada.por','Programa','NOMBRE DEL USUARIO']],on='Bitacora.registrada.por',how='left')

In [27]:
corte_nov['Programa'].isnull().sum()
# 1736
# sin match

1736

In [28]:
corte_nov.shape

(22392, 38)

In [29]:
corte_nov.Programa.fillna('Sin Identificar',inplace=True)

/tmp/ipykernel_6987/525378345.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  corte_nov.Programa.fillna('Sin Identificar',inplace=True)


### Limpieza coordenadas

In [30]:

def to_float(i):
    """
    el parámetro i es un del tipo str
    -----------------------------------
    La función to_float convierte a 'i' a tipo flotante
    -----------------------------------
    devuelve un valor del tipo float o None si no es posible transformalo directamente
    
    """
    try:
        if '°' in i: 
            i = None
            return i          # para coordenadas en formato grados, min, segundos
        else:
            i = re.sub(r"\\t",'',i)
            i = float(i)
            return i 
    except:
         i = None
    return i

In [31]:

def transform(row):
    """
    el parámetro row es un valor del tipo str
    ----------------------------------------------
    separa la cadena por comas
    depura aquellos que no cumplen ciertas condiciones asignandoles el valor de lista vacia
    transforma cada cadena de texto a una lista de valores del tipo float
    ------------------------------------------------
    Retorna coor la cual puede ser una lista con valores flotantes 
    o una lista vacía.

    """
    input = row.split(',')

    size_list = len(input)


    if size_list ==1:
        coor = None

    elif size_list ==2:
        
        try:
            # input.replace('\\t','')
            pre_punto = map(to_float,input)
            coor = list(pre_punto)
            if None in coor:
                coor = None
                return coor
            else:
                return coor
        except ValueError as e:
            print(e)
            return None
    elif size_list >2:
        not_point = re.split(r",0\\n\\n|,0|0\s-|,",row)
        if ("" in not_point):
            not_point.remove("")
            list_with_float = map(to_float,not_point)
            coor = list(list_with_float)
            return coor
        else:
            list_with_float = map(to_float,not_point)
            coor = list(list_with_float)
            return coor

In [32]:

def to_matrix(listado):
    """
    el parámetro listado es un dato del tipo lista
    -----------------------------------
    transforma una lista a un array de numpy
    asigna la forma del array
    -----------------------------------
    devuelve una matriz
    """
    n = len(listado)
    c = 2
    f =  n/c
    f = int(f)
    vector = np.array(listado)
    matriz = vector.reshape(f,c)
    return matriz

In [33]:
def geometry_apply(x):
    """
    el parámetro x es una lista
    ------------------------------------------
    de acuerdo a la longitud de la lista aplica una o dos funciones
    -----------------------------------------
    devuelve geometry que puede ser un objeto Point, Polygon o un valor None
    """

    n = x
    if len(n) == 2:
        # orden=arreglo(n)
        # p=sorted(n)
        # print(p)
        geometry = Point(n)
    elif len(n) >5 and len(n) %2 == 0:
        listado = to_matrix(n)
        geometry = Polygon(listado)
    else:
        geometry = None
    print(geometry)
    return geometry

In [34]:
def typeshape(x):
    """
    El parámetro x es un str
    ----------------------------------------
    De acuerdo al valor genera ciertas cadenas de texto
    ---------------------------------------
    retorna label que es un str
    """
    n=x
    if int(n) == 2:
        label = "Point"
    else:
        label = "Polygon"
    return label  

In [35]:
def to_geopandas_df(df,origin_file):
    """
    file_input recibe al archivo Sitios.txt
    -----------------------------------------------
    Lee el archivo
    genera columnas y aplica funciones
    crea 3 subsets distintos
    elimina columnas temporales y columna Coordenadas areas
    exportar los sets a un formato específico
    -----------------------------------------------
    genera los archivos de salida o un error
    """
    try:
        
         #read   
   
        sitiosbd = df.copy()
        #crear col geografica
        sitiosbd['info_geo'] = sitiosbd['Coordenadas.areas']
        #castear dato a str
        sitiosbd['info_geo']=sitiosbd['info_geo'].astype(str)
        #se aplica transform
        sitiosbd['info_geo']= sitiosbd['info_geo'].apply(transform)
        desechados=sitiosbd[sitiosbd['info_geo'].isnull()]
        sitiosbd_depurate=sitiosbd[sitiosbd['info_geo'].notna()]
        sitiosbd_depurate['n_vertices'] = sitiosbd_depurate['info_geo'].apply(lambda x:len(x))
        sitiosbd_depurate['geometry'] = sitiosbd_depurate['info_geo'].apply(geometry_apply)
        
        sitiosbd_depurate=sitiosbd_depurate[sitiosbd_depurate['geometry'].notna()]
        desechados_2=sitiosbd_depurate[sitiosbd_depurate['geometry'].isnull()]
        desechados = pd.concat([desechados_2,desechados],axis=0)
    
        desechados.to_csv(f'./errors_{origin_file}.csv', index=False)
        
        sitiosbd_depurate['type_shape'] = sitiosbd_depurate['n_vertices'].apply(typeshape)
        sitiosbd_polygon = sitiosbd_depurate[sitiosbd_depurate['type_shape']=='Polygon']
        sitiosbd_point = sitiosbd_depurate[sitiosbd_depurate['type_shape']=='Point']
        
        gdf_points_sitios = gpd.GeoDataFrame(sitiosbd_point,geometry='geometry',crs="EPSG:4326")
        gdf_polygons_sitios = gpd.GeoDataFrame(sitiosbd_polygon,geometry='geometry',crs="EPSG:4326")

        eliminar = ['Coordenadas.areas','info_geo','n_vertices','type_shape','Bitacora.registrada.por','Nombre.Productor']
        gdf_points_sitios=gdf_points_sitios.drop(columns=eliminar)
        gdf_polygons_sitios=gdf_polygons_sitios.drop(columns=eliminar)
        
        return gdf_points_sitios,gdf_polygons_sitios

    except FileNotFoundError as e:
        print(f'Por favor ejecute el script en el mismo directorio donde se encuentre el archivo archivo / df {e}')

In [36]:
pts, pols=to_geopandas_df(corte_nov,'nov_30_')

/tmp/ipykernel_6987/1264477346.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sitiosbd_depurate['n_vertices'] = sitiosbd_depurate['info_geo'].apply(lambda x:len(x))


POINT (16.517881 -96.96936)
POLYGON ((-97.07149555 16.94810472, -97.07170477 16.9482484, -97.07210173 16.94843569, -97.07227876 16.94851266, -97.07195958 16.94881285, -97.07155188 16.94893601, -97.07148214 16.94876667, -97.07144191 16.94848188, -97.07140972 16.94807393, -97.07149555 16.94810472))
POLYGON ((-97.86132604 16.42654657, -97.86124558 16.42691705, -97.86147893 16.42704568, -97.86159426 16.4268553, -97.86140651 16.42677812, -97.86144674 16.42658774, -97.86132604 16.42654657))
POLYGON ((-97.84471827 16.62592518, -97.84324305 16.62622331, -97.84336643 16.62647517, -97.84357028 16.62644433, -97.84465389 16.62616162, -97.84474509 16.6260691, -97.84475045 16.62596116, -97.84471827 16.62592518))
POINT (17.616504 -95.835015)
POINT (16.795797 -96.362524)
POINT (16.592316 -95.130252)
POINT (16.578958 -95.108567)
POINT (16.588988 -95.097938)
POINT (16.596626 -95.068992)
POINT (16.573862 -95.090777)
POINT (16.802577 -97.773596)
POINT (16.956042 -97.923046)
POINT (16.802919 -97.77371)
POI

/tmp/ipykernel_6987/1264477346.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sitiosbd_depurate['geometry'] = sitiosbd_depurate['info_geo'].apply(geometry_apply)


In [37]:
pts.shape
# (20334, 36)

(20334, 36)

In [38]:
pols.shape
# (1604, 36)

(1604, 36)

In [39]:
reg_oax = gpd.GeoDataFrame(gpd.read_file(regiones_oax_file,geometry='geometry',crs="EPSG:4326"))

/home/ubuntu/sefader/venv/lib/python3.10/site-packages/pyogrio/raw.py:196: RuntimeWarning: driver GeoJSON does not support open option GEOMETRY
  return ogr_read(
/home/ubuntu/sefader/venv/lib/python3.10/site-packages/pyogrio/raw.py:196: RuntimeWarning: driver GeoJSON does not support open option CRS
  return ogr_read(


In [40]:
pols.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1604 entries, 1 to 22391
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   ID.de.bitacora                  1604 non-null   object  
 1   Tipo.de.superficie              1604 non-null   object  
 2   ID.del.productor                1041 non-null   object  
 3   CURP                            1041 non-null   object  
 4   ID.parcela                      860 non-null    object  
 5   ID.del.sitio                    1604 non-null   object  
 6   Nombre.de.la.Parcela            1604 non-null   object  
 7   Estado.areas                    1604 non-null   object  
 8   Municipio.areas                 1603 non-null   object  
 9   Tipo.de.bitacora                1604 non-null   object  
 10  Anio.Bitacora                   1604 non-null   object  
 11  Ciclo                           1604 non-null   object  
 12  Regimen.hidrico 

In [41]:
#  poligonos dentro de oax
ds = gpd.sjoin(pols,reg_oax, how="inner", predicate='within')
# longitud polygons  (1604, 36)

In [42]:
ds.shape
# menos 28

(1576, 43)

In [43]:
# Todos los puntos (AI simplificadas) tienen las coordenadas invertidas
# pts.explore()

In [44]:
# Función para intercambiar el orden de latitud y longitud en cada Point
def swap_lat_lon(point):
    return Point(point.y, point.x)

In [45]:
pts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 20334 entries, 0 to 21529
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   ID.de.bitacora                  20334 non-null  object  
 1   Tipo.de.superficie              20334 non-null  object  
 2   ID.del.productor                566 non-null    object  
 3   CURP                            566 non-null    object  
 4   ID.parcela                      0 non-null      object  
 5   ID.del.sitio                    0 non-null      object  
 6   Nombre.de.la.Parcela            20334 non-null  object  
 7   Estado.areas                    20334 non-null  object  
 8   Municipio.areas                 20334 non-null  object  
 9   Tipo.de.bitacora                20334 non-null  object  
 10  Anio.Bitacora                   20334 non-null  object  
 11  Ciclo                           20334 non-null  object  
 12  Regimen.hidrico

In [46]:
pts['geometry'] = pts['geometry'].apply(swap_lat_lon)

In [47]:
# los puntos que caen dentro de oax

ds2 = gpd.sjoin(pts,reg_oax, how="inner", predicate='within')

In [48]:
ds2.shape
# de puntos 20334


(20044, 43)

In [49]:
ds2['Region'].unique()

array(['Sierra Sur', 'Papaloapan', 'Valles centrales', 'Istmo', 'Costa',
       'Sierra Norte', 'Cañada', 'Mixteca'], dtype=object)

In [50]:
ds2['Region'].replace({'Valles centrales':'Valles Centrales'},inplace=True)

/tmp/ipykernel_6987/2739909976.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ds2['Region'].replace({'Valles centrales':'Valles Centrales'},inplace=True)


In [51]:
ds['Region'].unique()

array(['Valles centrales', 'Costa', 'Sierra Sur', 'Cañada', 'Papaloapan',
       'Sierra Norte', 'Mixteca', 'Istmo'], dtype=object)

In [52]:
ds['Region'].replace({'Valles centrales':'Valles Centrales'},inplace=True)

/tmp/ipykernel_6987/3491809979.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ds['Region'].replace({'Valles centrales':'Valles Centrales'},inplace=True)


In [53]:
def replace_chracter(x):
    text = re.sub('_',' ',x)
    return text

In [54]:
ds['Anio.Bitacora'].unique()

array(['OI_2023_2024', 'PV_2024'], dtype=object)

In [55]:
ds2['Anio.Bitacora'].unique()

array(['OI_2023_2024', 'PV_2024'], dtype=object)

In [56]:
ds['Anio.Bitacora'] = ds['Anio.Bitacora'].apply(replace_chracter)

In [57]:
ds['Anio.Bitacora'].unique()

array(['OI 2023 2024', 'PV 2024'], dtype=object)

In [58]:
ds2['Anio.Bitacora'] = ds2['Anio.Bitacora'].apply(replace_chracter)

In [59]:
ds2['Anio.Bitacora'].unique()

array(['OI 2023 2024', 'PV 2024'], dtype=object)

In [60]:
dataframe_output = pd.concat([ds,ds2],ignore_index=True)

In [61]:
dataframe_output.shape

(21620, 43)

In [62]:
dataframe_output.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 21620 entries, 0 to 21619
Data columns (total 43 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   ID.de.bitacora                  21620 non-null  object  
 1   Tipo.de.superficie              21620 non-null  object  
 2   ID.del.productor                1580 non-null   object  
 3   CURP                            1580 non-null   object  
 4   ID.parcela                      853 non-null    object  
 5   ID.del.sitio                    1576 non-null   object  
 6   Nombre.de.la.Parcela            21620 non-null  object  
 7   Estado.areas                    21620 non-null  object  
 8   Municipio.areas                 21619 non-null  object  
 9   Tipo.de.bitacora                21620 non-null  object  
 10  Anio.Bitacora                   21620 non-null  object  
 11  Ciclo                           21620 non-null  object  
 12  Regimen.hi

In [63]:
ds2.to_file('../data/puntos_nov.geojson', driver='GeoJSON')
ds.to_file('../data/poligonos_nov.geojson', driver='GeoJSON')

In [64]:
dataframe_output.to_csv('../data/corte_clean_nov.csv')